## Analyse grammaticale avec NLTK et Spacy
On présente dans ce notebook une comparaison des fonctions de base de NLTK et Spacy pour effectuer une analyse grammaticale (part-of-speech tagging).

## NLTK
Tout d'abord, NLTK offrent plusieurs implémentations de POS taggers (Stanford, transformation-based, n-grams, etc.). Toutefois, on se limite ici à celui offert par la fonction pos-tag. L'analyse est faite au niveau de la phrase et on doit faire une tokeniser la phrase au préalable. Les étiquettes correspondent à celles du Penn Treebank.

In [1]:
import nltk

sent = "This is a test."
tokens = nltk.word_tokenize(sent)
tagged_tokens = nltk.pos_tag(tokens)
for token, tag in tagged_tokens:
    print(token, tag)

This DT
is VBZ
a DT
test NN
. .


## Spacy
Spacy est un logiciel NLP qui offre également un large éventail d'analyse de textes. La particularité de Spacy est que l'analyseur fait l'ensemble de ses analyses en une seule opération. Pour le POS tagging, on obtient les résultats de l'analyse en allant chercher soit l'attribut tag_ (étiquettes Penn Treebank) ou pos_ (étiquettes Universal Dependencies - UD) de chacun des jetons de la phrase. On présente ici quelques exemples pour l'anglais. Mais Spacy offre des modèles pour plusieurs langues, dont le français (voir ressource fr_core_web_sm).

Note: Dans les exemples sur le Web, on utilise souvent la variable nlp pour désigner l'instance d'analyseur obtenu en chargeant un modèle (spacy.load(model)). J'utilise plutôt le nom analyzer dans mes exemples, ce qui me semble un meilleur nommage.

In [2]:
import spacy

analyzer_en = spacy.load("en_core_web_sm")

doc = analyzer_en(sent)
for token in doc:
    print(token.text, token.tag_, token.pos_)

This DT DET
is VBZ AUX
a DT DET
test NN NOUN
. . PUNCT


## Attributs (features) de mots de Spacy
Spacy offre également différents attributs pour décrire la morphologie de chacun des jetons (dont le lemme obtenu par lemmatisation). Comme décrit dans la capsule vidéo sur les MEMM, ces informations sont utiles dans le POS tagging, entres autres pour l'étiquetage de mots inconnus. En voici un exemple.

In [3]:
import pandas as pd

sent = "CRJ-1000 planes were designed by Bombardier."
doc = analyzer_en(sent)
word_features = dict()
for token in doc:
    word_features[token.text] = [token.lemma_, token.shape_, token.prefix_, token.suffix_, 
                                 token.is_lower, token.is_upper, token.is_punct]
feature_names = ["Lemme", "Forme", "Prefixe", "Suffixe", "Minuscule", "Majuscule", "Ponctuation"]
df = pd.DataFrame.from_dict(word_features, orient='index', columns=feature_names)
display(df)

,Lemme,Forme,Prefixe,Suffixe,Minuscule,Majuscule,Ponctuation
CRJ-1000,CRJ-1000,XXX-dddd,C,000,False,True,False
planes,plane,xxxx,p,nes,True,False,False
were,be,xxxx,w,ere,True,False,False
designed,design,xxxx,d,ned,True,False,False
by,by,xx,b,by,True,False,False
Bombardier,Bombardier,Xxxxx,B,ier,False,False,False
.,.,.,.,.,False,False,True


## Comparaison NLTK-Spacy
Pour terminer, on compare ici dans les prochaines cellules les résultats obtenus par NLTK et Spacy sur quelques phrases.

In [5]:
import pandas as pd 

def compare_tagging(sentence):
    doc = analyzer_en(sentence)
    tokens = [token.text for token in doc]
    nltk_tagged_tokens = nltk.pos_tag(tokens)
    pos_tags = {
        'tokens': [token.text for token in doc], 
        'tags-nltk': [tag for token, tag in nltk_tagged_tokens],
        'tags-spacy': [token.tag_ for token in doc],
        'pos-spacy': [token.pos_ for token in doc] 
    }
    df = pd.DataFrame.from_dict(pos_tags, orient='index')
    display(df)

Souvent les 2 analyses concordent.



In [6]:
sent1 = "Oil falls 5% as economic outlook dims with rising virus cases."
compare_tagging(sent1)

,0,1,2,3,4,5,6,7,8,9,10,11,12
tokens,Oil,falls,5,%,as,economic,outlook,dims,with,rising,virus,cases,.
tags-nltk,NN,VBZ,CD,NN,IN,JJ,NN,NNS,IN,VBG,NN,NNS,.
tags-spacy,NN,VBZ,CD,NN,IN,JJ,NN,NNS,IN,VBG,NN,NNS,.
pos-spacy,NOUN,VERB,NUM,NOUN,SCONJ,ADJ,NOUN,NOUN,ADP,VERB,NOUN,NOUN,PUNCT


Certaines erreurs sont parfois fréquentes, dont l'ambiguïté entre le there existentiel (comme dans there is - "il y a") et l'adverbe de lieu (there = là).



In [7]:
sent2 = "Why is there a B.C. election?"
compare_tagging(sent2)

,0,1,2,3,4,5,6
tokens,Why,is,there,a,B.C.,election,?
tags-nltk,WRB,VBZ,RB,DT,NNP,NN,.
tags-spacy,WRB,VBZ,EX,DT,NNP,NN,.
pos-spacy,ADV,AUX,PRON,DET,PROPN,NOUN,PUNCT


Dans cet exemple, quelques petits problèmes avec NLTK pour les noms propres.



In [8]:
sent3 = "Trump and Beijing both hinted at blocking the Oracle-TikTok deal."
compare_tagging(sent3)

,0,1,2,3,4,5,6,7,8,9,10,11,12
tokens,Trump,and,Beijing,both,hinted,at,blocking,the,Oracle,-,TikTok,deal,.
tags-nltk,NN,CC,NNP,DT,VBD,IN,VBG,DT,NNP,:,NN,NN,.
tags-spacy,NNP,CC,NNP,DT,VBD,IN,VBG,DT,NNP,HYPH,NNP,NN,.
pos-spacy,PROPN,CCONJ,PROPN,DET,VERB,ADP,VERB,DET,PROPN,PUNCT,PROPN,NOUN,PUNCT


Il semble que NLTK n'a pas été entraîné pour les titres de bulletins de nouvelles comportant des majuscules.



In [9]:
sent4 = "Apple (AAPL) Precipitous Sell-Off Adds Fuel to Buffett Rumors"
compare_tagging(sent4)


,0,1,2,3,4,5,6,7,8,9,10,11,12
tokens,Apple,(,AAPL,),Precipitous,Sell,-,Off,Adds,Fuel,to,Buffett,Rumors
tags-nltk,NNP,(,NNP,),NNP,NNP,:,NNP,NNP,NNP,TO,NNP,NNS
tags-spacy,NNP,-LRB-,NNP,-RRB-,JJ,NN,HYPH,NN,VBZ,NNP,IN,NNP,NNP
pos-spacy,PROPN,PUNCT,PROPN,PUNCT,ADJ,NOUN,PUNCT,NOUN,VERB,PROPN,ADP,PROPN,PROPN


Et convertir la phrase en lettres minuscules ne règle pas complètement le problème.



In [10]:
 compare_tagging(sent4.lower()) 


,0,1,2,3,4,5,6,7,8,9,10,11,12
tokens,apple,(,aapl,),precipitous,sell,-,off,adds,fuel,to,buffett,rumors
tags-nltk,NN,(,NN,),JJ,SYM,:,RP,VBZ,NN,TO,VB,NNS
tags-spacy,NNP,-LRB-,NNP,-RRB-,JJ,NN,HYPH,NN,VBZ,NN,IN,NNP,NNS
pos-spacy,PROPN,PUNCT,PROPN,PUNCT,ADJ,NOUN,PUNCT,NOUN,VERB,NOUN,ADP,PROPN,NOUN


Un dernier exemple comportant quelques ambiguïtés dont l'étiquetage de verbes à particules en anglais (tighten up et settle in).



In [11]:
sent5 = "Tighten 'er up, settle in, Trudeau about to wreak havoc."
compare_tagging(sent5)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
tokens,Tighten,',er,up,",",settle,in,",",Trudeau,about,to,wreak,havoc,.
tags-nltk,NNP,POS,NN,RB,",",VB,IN,",",NNP,IN,TO,VB,NN,.
tags-spacy,VB,``,UH,RP,",",VB,RP,",",NNP,IN,TO,VB,NN,.
pos-spacy,VERB,PUNCT,INTJ,ADP,PUNCT,VERB,ADP,PUNCT,PROPN,ADP,PART,VERB,NOUN,PUNCT
